# Coupled basis to uncoupled basis
This notebook is for converting between states in the coupled and uncoupled bases. First import the necessary packages:

In [16]:
import sys
sys.path.append('../molecular-state-classes-and-functions/')

import numpy as np
import sympy
import pickle
import matplotlib.pyplot as plt
%matplotlib notebook

from classes import UncoupledBasisState,CoupledBasisState, State
from functions import sort_evals, distancematrix


with open("hamiltonians_symbolic.py", 'rb') as f:
    hamiltonians = pickle.load(f)
with open("Hff_alt_mat.py", 'rb') as f:
    Hff_alt_mat_m = pickle.load(f)["Hff_alt_mat"]
    
    
Jmax = sympy.Integer(6) # max J value in Hamiltonian
I_Tl = 1/2             # I1 in Ramsey's notation/
I_F  = 1/2             # I2 in Ramsey's notation

def ni_range(x0, x1, dx=1):
    # sanity check arguments
    if dx==0:
        raise ValueError("invalid parameters: dx==0")
    if x0>x1 and dx>=0:
        raise ValueError("invalid parameters: x0>x1 and dx>=0")
    if x0<x1 and dx<=0:
        raise ValueError("invalid parameters: x0<x1 and dx<=0")
        
    # generate range list
    range_list = []
    x = x0
    while x < x1:
        range_list.append(sympy.Number(x))
        x += dx
    return range_list

QN = [UncoupledBasisState(J,mJ,I_Tl,m1,I_F,m2)
      for J  in ni_range(0, Jmax+1)
      for mJ in ni_range(-J,J+1)
      for m1 in ni_range(-I_Tl,I_Tl+1)
      for m2 in ni_range(-I_F,I_F+1)
     ]

## Electrostatic lens state $| J = 2, m_J = 0> $
The state used in the molecular lens is $| J = 2, m_J = 0> $ with unknown $m_1$ and $m_2$. In the coupled basis these states are:

In [19]:
Fs = (1,2,2,3)
mFs = (-1,0,1)
F1s = (3/2,3/2,5/2,5/2)
J = 2
mJ = 0
I1 = 1/2
I2 = 1/2
m1s = np.arange(-I1,I1+1,1)
m2s = np.arange(-I2,I2+1,1)

for m1 in m1s:
    for m2 in m2s:
        data = []
        UncoupledBasisState(J,mJ,I1,m1,I2,m2).print_quantum_numbers()
        uncoupled_state = State([(1.0, UncoupledBasisState(J,mJ,I1,m1,I2,m2))])
        for i in range(0,len(Fs)):
            F = Fs[i]
            F1 = F1s[i]
            for mF in mFs:
                coupled_state = CoupledBasisState(F,mF,F1,J,I1,I2).transform_to_uncoupled()
                amp = uncoupled_state@coupled_state
                data.append((amp, CoupledBasisState(F,mF,F1,J,I1,I2)))
        print('=')
        State(data).print_state()
        print('')

|J = 2, mJ = 0, I1 = 1/2, m1 = -1/2, I2 = 1/2, m2 = -1/2>
=
+0.3162 x |F = 1, mF = -1, F1 = 3/2, J = 2, I1 = 1/2, I2 = 1/2>
+0.5477 x |F = 2, mF = -1, F1 = 3/2, J = 2, I1 = 1/2, I2 = 1/2>
+0.4472 x |F = 2, mF = -1, F1 = 5/2, J = 2, I1 = 1/2, I2 = 1/2>
+0.6325 x |F = 3, mF = -1, F1 = 5/2, J = 2, I1 = 1/2, I2 = 1/2>

|J = 2, mJ = 0, I1 = 1/2, m1 = -1/2, I2 = 1/2, m2 = 1/2>
=
-0.4472 x |F = 1, mF = 0, F1 = 3/2, J = 2, I1 = 1/2, I2 = 1/2>
+0.4472 x |F = 2, mF = 0, F1 = 3/2, J = 2, I1 = 1/2, I2 = 1/2>
-0.5477 x |F = 2, mF = 0, F1 = 5/2, J = 2, I1 = 1/2, I2 = 1/2>
+0.5477 x |F = 3, mF = 0, F1 = 5/2, J = 2, I1 = 1/2, I2 = 1/2>

|J = 2, mJ = 0, I1 = 1/2, m1 = 1/2, I2 = 1/2, m2 = -1/2>
=
-0.4472 x |F = 1, mF = 0, F1 = 3/2, J = 2, I1 = 1/2, I2 = 1/2>
-0.4472 x |F = 2, mF = 0, F1 = 3/2, J = 2, I1 = 1/2, I2 = 1/2>
+0.5477 x |F = 2, mF = 0, F1 = 5/2, J = 2, I1 = 1/2, I2 = 1/2>
+0.5477 x |F = 3, mF = 0, F1 = 5/2, J = 2, I1 = 1/2, I2 = 1/2>

|J = 2, mJ = 0, I1 = 1/2, m1 = 1/2, I2 = 1/2, m2 = 1/2>
=
+

However, I think whatever hyperfine state we have entering the lens is the one that will come out simply due to adiabatic following (but maybe not).

Below calculating the state content of the possible coupled basis states that could enter the lens

In [20]:
#Set the possible values of the quantum numbers:
Fs = (1,2,2,3)
mFs = (-1,0,1)
F1s = (3/2,3/2,5/2,5/2)
J = 2
I1 = 1/2
I2 = 1/2

for i in range(0,len(Fs)):
    F = Fs[i]
    F1 = F1s[i]
    for mF in mFs:
        state = CoupledBasisState(F,mF,F1,J,I1,I2)
        state.print_quantum_numbers()
        state.transform_to_uncoupled().print_state()
        print("")

|F = 1, m_F = -1, F1 = 3/2, J = 2, I1 = 1/2, I2 = 1/2>
+0.7746 x |J = 2, mJ = -2, I1 = 1/2, m1 = 1/2, I2 = 1/2, m2 = 1/2>
-0.3873 x |J = 2, mJ = -1, I1 = 1/2, m1 = -1/2, I2 = 1/2, m2 = 1/2>
-0.3873 x |J = 2, mJ = -1, I1 = 1/2, m1 = 1/2, I2 = 1/2, m2 = -1/2>
+0.3162 x |J = 2, mJ = 0, I1 = 1/2, m1 = -1/2, I2 = 1/2, m2 = -1/2>

|F = 1, m_F = 0, F1 = 3/2, J = 2, I1 = 1/2, I2 = 1/2>
+0.5477 x |J = 2, mJ = -1, I1 = 1/2, m1 = 1/2, I2 = 1/2, m2 = 1/2>
-0.4472 x |J = 2, mJ = 0, I1 = 1/2, m1 = -1/2, I2 = 1/2, m2 = 1/2>
-0.4472 x |J = 2, mJ = 0, I1 = 1/2, m1 = 1/2, I2 = 1/2, m2 = -1/2>
+0.5477 x |J = 2, mJ = 1, I1 = 1/2, m1 = -1/2, I2 = 1/2, m2 = -1/2>

|F = 1, m_F = 1, F1 = 3/2, J = 2, I1 = 1/2, I2 = 1/2>
+0.3162 x |J = 2, mJ = 0, I1 = 1/2, m1 = 1/2, I2 = 1/2, m2 = 1/2>
-0.3873 x |J = 2, mJ = 1, I1 = 1/2, m1 = -1/2, I2 = 1/2, m2 = 1/2>
-0.3873 x |J = 2, mJ = 1, I1 = 1/2, m1 = 1/2, I2 = 1/2, m2 = -1/2>
+0.7746 x |J = 2, mJ = 2, I1 = 1/2, m1 = -1/2, I2 = 1/2, m2 = -1/2>

|F = 2, m_F = -1, F1 = 3/2

## EDM measurement state
For the EDM measurement state we can use (I believe) any of the $m_J = \pm 1$, $m_2 = \pm\frac{1}{2}$ states. The goal is to simply get the maximal number of molecules in the correct state.